# 北京房价预测

问题：利用Kaggle上2011-2018年初的北京链家房价数据进行未来房价的预测

## 数据预处理

### 数据清洗

使用pandas读入数据，并获取数据集长度。

In [ ]:
import pandas as pd
data =pd.read_csv('../input/lianjia/new.csv',encoding = 'GB2312',low_memory=False)
print(data.shape[0])

In [ ]:
data

查看各列数据缺失行数。

In [ ]:
data.isnull().sum(axis=0)

统计发现DOM缺失行数约占50%，需补全缺失数据；而buildingType、elevator等列缺失行数较少可进行删除处理，对总体信息含量的影响较小。

In [ ]:
data['DOM'].fillna(data['DOM'].median(),inplace=True)
data=data.dropna(axis=0, how='any')

删除数据后对数据缺失情况进行检查。

In [ ]:
data.isnull().sum(axis=0)

此时数据集共316448条。

In [ ]:
print(data.shape[0])

处理floor字段，将该列的楼层数取出；

In [ ]:
data['floor']=data['floor'].map(lambda x:x.split(' ')[1]).astype('int64')

处理constructionTime字段，发现有中文标注为“未知”的数据，进行删除；

In [ ]:
data['constructionTime'].value_counts()

In [ ]:
data=data[~data['constructionTime'].isin(['未知'])]

### 数据类型修改

对部分字段的数据类型进行处理，设置为int64；

In [ ]:
data['livingRoom']=data['livingRoom'].astype('int64')
data['drawingRoom']=data['drawingRoom'].astype('int64')
data['bathRoom']=data['bathRoom'].astype('int64')
data['constructionTime']=data['constructionTime'].astype('int64')
data['buildingType']=data['buildingType'].astype('int64')


将tradeTime转换为datetime类型。

In [ ]:
data['tradeTime'] = pd.to_datetime(data['tradeTime'])
data.info()

经上述处理过后，数据总共还有297701条

In [ ]:
print(data.shape[0])

## 多元线性回归

### 相关关系

选取DOM,totalprice,price,followers,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,constructionTime,ladderRatio,communityAverage等定量数据求相关系数，并作热力图。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='whitegrid', context='notebook')
cols = ['DOM','totalPrice','price','followers','square','livingRoom','drawingRoom','kitchen','bathRoom','floor','constructionTime','ladderRatio','communityAverage']
cm = np.corrcoef(data[cols],rowvar=0)
plt.figure(figsize=(10, 10))
sns.set(font_scale=1.5)
hm = sns.heatmap(cm,cbar=True,annot=True,square=True,fmt='.2f',annot_kws={'size': 15},yticklabels=cols,xticklabels=cols)
plt.show()

- 从图中可以发现房价与每平米均价、面积、大厅、卫生间、小区均价的呈正相关关系且相关系数较高，均接近0.5；
- ladderRatio与其他变量均不相关；
- 房屋面积与大厅、卧室、卫生间之间的相关系数均较高，分别为0.72、0.62、0.73；
- 每平米均价与小区均价呈正相关关系且相关系数为0.68，符合认知习惯；

### 数据透视

这一部分主要针对相同交易时间（按月份）的每平米均价及交易量做图表，通过观察发现数据中有部分月份不连续，进行了删除，最终数据集中只剩下了交易时间在2010年1月到2018年1月之间的数据。

按照月频率对数据的price列求平均并进行计数，删除任意列有空值（主要针对price）的数据。

In [ ]:
avgprice=pd.DataFrame(data=data.resample('1M',on='tradeTime')['price'].mean())
avgprice['count']=data.resample('1M',on='tradeTime')['id'].count()
avgprice=avgprice.dropna(axis=0, how='any')
avgprice['count']=avgprice['count'].astype('int64')

观察索引发现存在不连续的月份数据，进行删除。

In [ ]:
avgprice.index

In [ ]:
avgprice=avgprice[4:]

画折线图观察趋势；

In [ ]:
avgprice.plot()

针对每月交易量做气泡图（圆圈越大代表交易量越大），与每平米均价所做折线图进行复合。

In [ ]:
import matplotlib.dates as mdate
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(25,16),dpi=100)
ax = plt.gca()
ax.xaxis.set_major_formatter(mdate.DateFormatter('%Y-%m'))
plt.xticks(pd.date_range(avgprice.index[0],avgprice.index[-1],freq='10M'))
plt.plot(avgprice.index,avgprice['price'],color="steelblue", linewidth=10, alpha=.25)
plt.scatter(avgprice.index, avgprice['price'],s=avgprice['count']/10,color="black",alpha=.75)
plt.show()

从上图中可以发现2015年数据较为平缓，这说明该年份房价受政策、央行贷款利率等宏观经济因素影响较小，可用于研究一般特征对房价的影响。

故选取2015年5月数据，针对每平米价格及位置做地理热力图。经观察，发现北京房价存在以天安门为中心向四周逐渐降低的趋势，故通过经纬度计算增加特征数据列center-distance描述房屋到天安门的距离，单位为千米。

In [ ]:
data=data[data['tradeTime']>'2010']
heat_data=data[data['tradeTime']>'2015-05']
heat_data=heat_data[heat_data['tradeTime']<'2015-06']

In [ ]:
max_price=heat_data['price'].max()

In [ ]:
dict={}
dict_list=heat_data.apply(lambda x:{x['id']:[x['Lng'],x['Lat']]},axis=1).to_list()
for item in dict_list:
    dict.update(item)

In [ ]:
values=heat_data.apply(lambda x:(x['id'],x['price']),axis=1).to_list()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install pyecharts
!pip install echarts-china-provinces-pypkg

In [ ]:
from pyecharts.charts import Geo
from pyecharts import options as opts
from pyecharts.globals import ChartType
import json
# with open('../input/test_data.json', 'w', encoding='utf-8') as json_file:
#     json_file.write(json.dumps(dict))
geo=Geo()
geo.add_schema(maptype= "北京",label_opts=opts.LabelOpts(is_show=False),center=[116.2317, 39.5427])
geo.add_coordinate_json(json_file='../input/locjson/test_data.json')
geo.add('热力图',values,label_opts=opts.LabelOpts(is_show=False),is_large=True,progressive_threshold=10000)
geo.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_ = max_price,split_number=8),
        title_opts=opts.TitleOpts(title="Geo-HeatMap"),
    )

In [ ]:
geo.render_notebook()

根据数据集经纬度进行房屋到天安门距离计算，作为新特征加入数据集

In [ ]:
from math import sqrt
from math import cos
from math import sin
import math

def rad(d):
    return d * math.pi / 180.0
 
def getDistance(lat1, lng1, lat2, lng2):
    EARTH_REDIUS = 6378.137
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(sin(a/2), 2) + cos(radLat1) * cos(radLat2) * math.pow(sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s

In [ ]:
data['center-distance']=data.apply(lambda x :getDistance(x['Lat'],x['Lng'], 39.5427,116.2317),axis=1)

### 回归模型

选取特征指标，对房子的总价进行回归拟合；选取所有数值型数据，过程中发现renovationCondition、buildingStructure、buildingType、district属于定性变量且种类超过2种；

In [ ]:
feature_cols=['DOM','followers','square','livingRoom','drawingRoom','kitchen','bathRoom','floor','constructionTime','elevator','ladderRatio','communityAverage','fiveYearsProperty','subway','center-distance']

定义方法对上述四种定性变量进行处理，以其随机的一个取值（index为2）作为默认值，防止出现共线性；

In [ ]:
def classify_values(data,column_name):
    temp=pd.DataFrame(data[column_name].value_counts())
    
    for index in temp.index: 
        if index==2:
            continue
        data[column_name+str(index)]=data[column_name].apply(lambda x: 1 if x==index else 0)
        feature_cols.append(column_name+str(index))

In [ ]:
classify_values(data,'renovationCondition')
classify_values(data,'buildingStructure')
classify_values(data,'buildingType')
classify_values(data,'district')

In [ ]:
data['tradeTime'].value_counts

使用train_test_split方法将因变量和自变量划分为训练集、测试集，默认比例为3:1，使用scikit-learn构建线性回归模型，进行训练。

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from scipy import stats
X = data[feature_cols]
y = data[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
linreg = LinearRegression(fit_intercept=True)
linreg.fit(X_train, y_train)

模型拟合优度及各变量参数值、p值如下图所示。

In [ ]:
import statsmodels.api as sm
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

### 模型检验

针对测试集进行预测，并计算MSE、RMSE；

In [ ]:
y_pred = linreg.predict(X_test)

In [ ]:
from sklearn import metrics
print ("MSE:",metrics.mean_squared_error(y_test, y_pred))
print ("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

### 讨论

在所有数据（训练集、测试集）上进行预测，并作图观察。

In [ ]:
y_avgreg=pd.DataFrame(data=linreg.predict(X))

此步应注意进行索引重拍，否则可能会出现因预测结果和交易时间不对应而产生作图的偏差。

In [ ]:
data.index = range(len(data))
y_avgreg.columns=['regPrice']
y_avgreg['tradeTime']=data['tradeTime']
y_avgreg

In [ ]:
y_avgreg=pd.DataFrame(data=y_avgreg.resample('1M',on='tradeTime')['regPrice'].mean())

y_avgreg.head()

In [ ]:
y_truth=pd.DataFrame(data=data.resample('1M',on='tradeTime')['price'].mean())
y_truth

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(25,16),dpi=100)
ax = plt.gca()
ax.xaxis.set_major_formatter(mdate.DateFormatter('%Y-%m'))
plt.xticks(pd.date_range(y_truth.index[0],y_truth.index[-1],freq='12M'))
plt.plot(y_truth.index,y_truth['price'],color="steelblue",linewidth=10, alpha=.45)
plt.plot(y_avgreg.index, y_avgreg['regPrice'],color="black",linewidth=10,alpha=.75)
plt.show()

## 模型调优

缩小数据范围，2015年价格变动较小，取2015年所有数据；

去除掉buildingStructure、renovationCondition中缺乏解释性的类

In [ ]:
data=data[~data['buildingStructure'].isin([1])]
data=data[~data['renovationCondition'].isin([1])]
data_2015=data[data['tradeTime']>'2015']
data_2015=data_2015[data_2015['tradeTime']<'2016']
data_2015.index=range(len(data_2015))

使用RFE方法对定量特征进行选择，最终发现当特征全选时，回归结果R^2较高，故参数n_features_to_select=15

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
X = data_2015[feature_cols[:15]]
y = data_2015[['price']]
selector=RFE(estimator=LinearRegression(), n_features_to_select=15)

In [ ]:
selector.fit_transform(X, y)
selector.get_support()

In [ ]:
features=[]
for index,value in enumerate(selector.get_support()):
    if value==True:
        features.append(feature_cols[index])
features.append(feature_cols[15])
features.append(feature_cols[17])
features=features+feature_cols[18:22]
features=features+feature_cols[23:]
X = data_2015[features]

去除在回归过程中参数检验不显著的变量

In [ ]:
X.drop(['DOM'],axis=1,inplace=True)
X.drop(['buildingStructure6'],axis=1,inplace=True)
X.drop(['buildingStructure4'],axis=1,inplace=True)
X.drop(['buildingStructure5'],axis=1,inplace=True)
X.drop(['district6'],axis=1,inplace=True)
X.drop(['district7'],axis=1,inplace=True)
X.drop(['district12'],axis=1,inplace=True)
X.drop(['district5'],axis=1,inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)
linreg = LinearRegression(fit_intercept=True)
linreg.fit(X_train, y_train)
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())